In [12]:
from finpy_main import DataBaseClient


import matplotlib.pyplot as plt                                            #type: ignore
import pandas as pd                                                        #type: ignore


from sklearn.linear_model import LogisticRegression                        #type: ignore
from sklearn.metrics import classification_report, confusion_matrix        #type: ignore
from sklearn.model_selection import TimeSeriesSplit                        #type: ignore
import numpy as np                                                         #type: ignore

In [2]:
db_snp = DataBaseClient('snp50_data.db')
db_econ = DataBaseClient('econ_data.db')

In [3]:
# db_snp.data_query("select distinct ticker from pricing_data")

In [4]:
gdp = db_econ.data_query("select * from all_data where ticker='GDP'")
unemp = db_econ.data_query("select * from all_data where ticker='UNRATE'")

this is where i forward fill economic data fro the days that it has been the most recent data.

In [5]:
visa = db_snp.data_query("select * from pricing_data where ticker='V'")

In [6]:
v_dates = visa['date']

In [7]:
gdp_sort = gdp.sort_values('date')
gdp_sort = gdp_sort.set_index('date').reindex(v_dates, method='ffill').reset_index()
gdp_sort

,date,value,ticker
0,2024-06-17,28269.174,GDP
1,2024-06-14,28269.174,GDP
2,2024-06-13,28269.174,GDP
3,2024-06-12,28269.174,GDP
4,2024-06-11,28269.174,GDP
...,...,...,...
4085,2008-03-26,14706.538,GDP
4086,2008-03-25,14706.538,GDP
4087,2008-03-24,14706.538,GDP
4088,2008-03-20,14706.538,GDP


In [8]:
v_with_gdp = pd.merge(visa, gdp_sort, on='date', how='left')
v_with_gdp.rename(columns={'value':'gdp'}, inplace=True)
v_with_gdp.drop(columns=['ticker_x','ticker_y'], inplace=True)
v_with_gdp

,date,open,high,low,close,volume,change,percent_change,vwap,gdp
0,2024-06-17,269.00,271.83,269.00,271.17,9255196,2.1700,0.8067,270.2500,28269.174
1,2024-06-14,270.10,271.55,269.27,270.66,8537260,0.5600,0.2073,270.3950,28269.174
2,2024-06-13,270.32,271.86,269.32,271.19,7361158,0.8700,0.3218,270.6725,28269.174
3,2024-06-12,276.77,277.60,269.90,270.32,7536636,-6.4500,-2.3300,273.6475,28269.174
4,2024-06-11,274.54,274.86,272.22,274.67,6244586,0.1300,0.0474,274.0725,28269.174
...,...,...,...,...,...,...,...,...,...,...
4085,2008-03-26,15.68,16.12,15.39,15.99,43139420,0.3075,1.9800,15.7950,14706.538
4086,2008-03-25,15.15,16.06,14.96,15.81,87165184,0.6675,4.3600,15.4950,14706.538
4087,2008-03-24,16.81,16.93,14.75,14.93,149694976,-1.8800,-11.1800,15.8550,14706.538
4088,2008-03-20,14.60,16.25,14.38,16.09,198984960,1.4900,10.2100,15.3300,14706.538


In [14]:
v_with_gdp['tom_close_delta'] = v_with_gdp['change'].shift(1)
v_with_gdp['tom_close_bin'] = (v_with_gdp['tom_close_delta'] >= 0).astype(int)
v_with_gdp[1::]

,date,open,high,low,close,volume,change,percent_change,vwap,gdp,tom_close_delta,tom_close_bin
1,2024-06-14,270.10,271.55,269.27,270.66,8537260,0.5600,0.2073,270.3950,28269.174,2.1700,1
2,2024-06-13,270.32,271.86,269.32,271.19,7361158,0.8700,0.3218,270.6725,28269.174,0.5600,1
3,2024-06-12,276.77,277.60,269.90,270.32,7536636,-6.4500,-2.3300,273.6475,28269.174,0.8700,1
4,2024-06-11,274.54,274.86,272.22,274.67,6244586,0.1300,0.0474,274.0725,28269.174,-6.4500,0
5,2024-06-10,278.14,278.56,273.38,275.04,5214002,-3.1000,-1.1100,276.2800,28269.174,0.1300,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4085,2008-03-26,15.68,16.12,15.39,15.99,43139420,0.3075,1.9800,15.7950,14706.538,-0.2850,0
4086,2008-03-25,15.15,16.06,14.96,15.81,87165184,0.6675,4.3600,15.4950,14706.538,0.3075,1
4087,2008-03-24,16.81,16.93,14.75,14.93,149694976,-1.8800,-11.1800,15.8550,14706.538,0.6675,1
4088,2008-03-20,14.60,16.25,14.38,16.09,198984960,1.4900,10.2100,15.3300,14706.538,-1.8800,0


In [15]:
features = ['open', 'high', 'low', 'close', 'volume', 'change', 'percent_change', 'vwap', 'gdp']
X = v_with_gdp[features]
y = v_with_gdp['tom_close_bin']

model = LogisticRegression(max_iter=1000)
tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print("="*50)

Confusion Matrix:
[[  3 325]
 [ 12 341]]

Classification Report:
              precision    recall  f1-score   support

           0       0.20      0.01      0.02       328
           1       0.51      0.97      0.67       353

    accuracy                           0.51       681
   macro avg       0.36      0.49      0.34       681
weighted avg       0.36      0.51      0.36       681

Confusion Matrix:
[[  0 306]
 [  0 375]]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       306
           1       0.55      1.00      0.71       375

    accuracy                           0.55       681
   macro avg       0.28      0.50      0.36       681
weighted avg       0.30      0.55      0.39       681

Confusion Matrix:
[[  0 334]
 [  0 347]]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       334
           1       0.51      1.00      0.6

/Users/gavinfancher/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gavinfancher/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gavinfancher/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu